In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib qt5


In [2]:
import h5py
import numpy as np
from skimage import io
from scipy.signal import find_peaks
import matplotlib.pyplot as plt

In [3]:
f = h5py.File('/home/pk/Documents/test.hdf5', 'r') 

In [6]:
f['stack'][1]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float16)

In [11]:
f.create_dataset("stack", (100, 1504, 36), dtype='float16', compression='gzip')

<HDF5 dataset "stack": shape (100, 1504, 36), type "<f2">

In [12]:
f['stack'][10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float16)

In [24]:
with h5py.File('/home/pk/Documents/test.hdf5', 'r') as f:
    f['stack'][10] = np.random.randn(1504, 36)

In [ ]:
f['stack']

In [3]:
def findBarcodesAndChannels(image, parameters = { 'minChannelLength': 200, 'minPeaksDistance' : 25, 
                    'barcodeWidth' : 48, 'channelsPerBlock': 21, 'plateauSize':15}):
    
    hist = np.sum(image, axis = 0) > parameters['minChannelLength']

    peaks, _ = find_peaks(hist, distance=parameters['minPeaksDistance'], plateau_size=parameters['plateauSize'])
    
    indices_with_larger_gaps = np.where(np.ediff1d(peaks) > parameters['barcodeWidth'])[0]
    
    locations_before_barcode = peaks[indices_with_larger_gaps]
    locations_after_barcode = peaks[indices_with_larger_gaps + 1]
    
    locations_barcode = np.rint(np.mean((locations_before_barcode,
                                        locations_after_barcode), axis = 0)).astype('int')
    
    num_barcodes = len(locations_barcode)
    # there are 5 barcodes seen in the image
    if num_barcodes == 5:
        # count the number of channels before the first barcode and after the 
        # last barcode and include them upto numChannels channels
        y_channels = []
        
        # channels before first barcode
        indices_before_first = np.where(peaks < locations_barcode[0])[0]
        y_channels.extend(list(peaks[indices_before_first]))
        
        for i in range(num_barcodes):
            indices = np.where(np.logical_and(peaks > locations_barcode[i-1],
                                             peaks < locations_barcode[i]))[0]
            y_channels.extend(list(peaks[indices]))
            
        # number of channels to count after the last
        number_to_include = parameters['channelsPerBlock'] - len(indices_before_first)
        indices_after_last = np.where(peaks > locations_barcode[-1])[0]
        y_channels.extend(list(peaks[indices_after_last][:number_to_include]))
        
    elif num_barcodes == 6:
        y_channels = []
        # count only the channels between barcodes and 
        # grab the (x, y) locations to cut,
        # x will be the top of the channel, row number
        # y will be the peak picked up in the histogram, between the barcodes
        # count 21 channels after calculating
        for i in range(num_barcodes):
            indices = np.where(np.logical_and(peaks > locations_barcode[i-1],
                                             peaks < locations_barcode[i]))[0]
            #if len(indices) == 21:
            # all good pick them up
            y_channels.extend(list(peaks[indices]))   
        
    else:
        # detection failure, since it is ambiguous skipp the position
        y_channels = []
        sys.stdout.write(f"Detection failure, {num_barcodes} detected\n")
        sys.stdout.flush()
    # locations of the barcode and locations of channels to cut.
    return locations_barcode, y_channels

In [4]:
i = 0
channelSegImageFile = '/home/pk/Documents/realtimeData/newAnalysisData/1/channelSegmentation/0.tiff'
channelSegImage = io.imread(channelSegImageFile)
locations_barcode, locations_channels = findBarcodesAndChannels(channelSegImage)
phaseFilename = '/home/pk/Documents/realtimeData/newAnalysisData/1/phaseFullImage/' + str(i) + '.tiff'
phaseImg = io.imread(phaseFilename)

In [12]:
saveDir = '/home/pk/Documents/realtimeData/newAnalysisData/2/oneMMChannelPhase/'


for i, location in enumerate(locations_channels,0):
    filename = saveDir + str(i)
    with h5py.File(saveDir + str(i) + '.hdf5', 'w') as f:
        f.create_dataset("channels", (400, 1504, 36), dtype='float16', compression='gzip')
        f['channels'][0] = phaseImg[:, 
                                          location - 18 : location + 18]

In [13]:
with h5py.File(saveDir + '0.hdf5', 'w') as f:
    

In [7]:
plt.imshow(data.astype('float32'))

In [8]:
phaseFilename

'/home/pk/Documents/realtimeData/newAnalysisData/1/phaseFullImage/0.tiff'

In [9]:
phaseImage = io.imread(phaseFilename)

In [10]:
phaseImage.dtype

dtype('float16')

In [11]:
plt.imshow(phaseImage.astype('float32'))

In [22]:
f = h5py.File('/home/pk/Documents/realtimeData/newAnalysisData/4/oneMMChannelCellSeg/0.hdf5', 'r')

In [25]:
list(f.keys())

[]

In [ ]:
f